In [15]:
import sys
import os
sys.path.insert(0, os.path.abspath('../src'))

In [111]:
import numpy as np
from tqdm.notebook import trange
from tqdm.notebook import tqdm

import model
import model_parameters as MP
from docplex.mp.model import Model

## Mathematical Model

#### Sets
- C	Set of campaigns.
- U	Set of customers.
- H	Set of channels
- D	Set of planning days.
- I	Set of quota categories.
- P	Set of priority categories.


In [17]:
C = 5 # number of campaigns
U = 100 # number of customers.
H = 3 # number of channels.
D = 7 # number of planning days.
I = 3 # number of quota categories.
P = 10 # number of priority categories.

In [112]:
print(f"number of campaigns {C}") #70 -camp.
print(f"number of customers {U}") #20.000.000 non-distinct
print(f"number of channels {H}") #6-7 h. --Drop
print(f"number of planning days {D}")
print(f"number of quota categories {I}")
print(f"number of priority categories {P}")
mdl = Model(name='Campaign Optimization')

number of campaigns 5
number of customers 100
number of channels 3
number of planning days 7
number of quota categories 3
number of priority categories 10


#### Parameters

##### - eligibility
$$
e_{cu}\left\{\begin{array}\\
        1 & \mbox{if }  customer\ u\ is\ eligible\ for\ campaign\ c\\
        0 & \mbox{otherwise } \\
    \end{array}
\right.
$$

In [19]:
e_cu = np.random.choice(2,(C, U)) #e_cu = np.ones((C, U), dtype='int8')
e_cu

array([[0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
        1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1,
        0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1,
        0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1,
        0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0,
        1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0,
        0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1,
        1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0,
        0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0],
       [1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1,
        0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0,
        1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1,
        0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1,
        0, 1, 0, 0, 0,

##### - quota categories
$$
q_{ic}\left\{\begin{array}\\
        1 & \mbox{if }  campaign\ c\ is\ a\ i^{th} type\ quota\ category\ campaign\ \\
        0 & \mbox{otherwise } \\
    \end{array}
\right.
$$

In [20]:
q_ic = np.random.choice(2, (I,C)) #q_ic = np.zeros((I,C), dtype='int8')
q_ic

array([[1, 0, 0, 1, 0],
       [0, 1, 1, 1, 1],
       [1, 0, 0, 0, 1]])

##### - priority categories
$$r_{cp}: Priority\ value\ of\ campaign\ c\ regarding\ priority\ type\ p\$$

In [21]:
r_p = np.random.choice(100, P) #r_p = np.ones(P, dtype='int8')
rp_c = np.array([r_p[r] for r in np.random.choice(P, C)])

rp_c

array([73, 38, 74, 22,  5])

##### - blokage
$$b: Communication\ limit\ per\ person\ for\ the\ whole\ period\$$

In [22]:
b = 7
b

7

##### - daily blokage
$$k: Communication\ limit\ per\ person\ at\ each\ day\$$

In [23]:
k = 3
k

3

##### - campaign blockage
$$l_c: Communication\ limit\ per\ person\ for\ campaign\ c\$$

In [24]:
l_c = np.random.choice([2,3,4],C)
l_c

array([4, 3, 2, 4, 3])

##### - quota limitations daily/weekly
$$
m_i: Communication\ limit\ per\ person\ for\ i^{th}\ category\
$$
$$
n_i: Communication\ limit\ per\ person\ for\ i^{th}\ category\ each\ day\
$$

In [25]:
m_i = np.random.choice([4,3,5],I)#m_i = np.ones((I), dtype='int8')*10
n_i = np.random.choice([1,3,2],I)#n_i = np.ones((I), dtype='int8')*10
(m_i, n_i)

(array([4, 5, 3]), array([3, 2, 2]))

#### - capacity for channel
$$
t_{h,d}: Capacity\ for\ channel\ h\ at\ day\ d.\
$$

In [26]:
t_hd = np.random.choice([U*.7, U*.6, U*.5], (H, D))
t_hd

array([[70., 50., 70., 70., 70., 60., 60.],
       [50., 50., 70., 60., 70., 60., 50.],
       [60., 60., 50., 70., 50., 60., 60.]])

# Model

#### Variables
$$
X_{cuhd}\left\{\begin{array}\\
        1 & \mbox{if } Campaign\ c\ will\ be\ sent\ to\ customer\ u\ through\ Channel\ h\ at\ Day\ d \\
        0 & \mbox{otherwise } \\
    \end{array}
\right.
$$

## Maximize
$$\sum_{p \in P}\sum_{c \in C}\sum_{u \in U}\sum_{h \in H}\sum_{d \in D}\,X_{cuhd}\ r_{cp}$$

##### Binary variable (10)
$$
X_{cuhd} \in \{ 1,0 \},\hspace{35pt} \forall c \in C ,\forall u \in U,\forall d \in D, \forall h \in H
$$

In [72]:
X_cuhd = np.zeros((C,U,H,D), dtype='int')

## subject to

##### - eligibility (2)

$$
X_{cuhd}  \leq e_{cu},\hspace{35pt} \forall h \in H,\forall d \in D
$$

##### - use one channel (3)
$$
\sum_{h}X_{cuhd} \le 1,\hspace{35pt} \forall c \in C \, \forall u \in U,\forall d \in D
$$

##### - weekly communication limitation (4)
$$
\sum_{h \in H}\sum_{c \in C}\sum_{d \in D} X_{cuhd}\le b,\hspace{35pt} \forall u \in U
$$

##### - daily communication limitation (5)
$$
\sum_{h \in H}\sum_{c \in C} X_{cuhd}\le k,\hspace{35pt} \forall u \in U, \forall d \in D
$$

##### - campaign communication limit(6)
$$
\sum_{d \in D}\sum_{h \in H} X_{cuhd}\le l_c,\hspace{35pt} \forall c \in C,\forall u \in U;
$$

##### - weekly quota(7)
$$
\sum_{d \in D}\sum_{h \in H}\sum_{c \in C}{X_{cuhd} q_{ic}}\le m_i,\hspace{35pt} \forall u \in U, \forall i \in I
$$

##### - daily quota(8)
$$
\sum_{h \in H}\sum_{c \in C}{X_{cuhd} q_{ic}}\le n_i,\hspace{35pt} \forall u \in U,\, \forall d \in D, \forall i \in I
$$

##### Channel capacity (9)
$$
\sum_{c \in C}\sum_{u \in U}{X_{cuhd}}\le t_{hd},\hspace{35pt} \forall d \in D,\, \forall h \in H
$$

In [73]:
class Solution:
    c_i = 0
    u_i = 1
    h_i = 2
    d_i = 3

    def __init__(self, name: str):
        self.name = name

    def eligibility(self, e_cu, X, c, u, h, d):
        return X[c,u,h,d]<=e_cu[c,u]
    def weekly_limitation(self, b, X, u):
        return X[:,u,:,:].sum() <= b
    def weekly_limitation_rh(self, b, X, s, u, f_d):
#        print(f"X[:,{u},:,:{f_d}].sum() + s[:,{u},:,{f_d}:].sum() <={b}")
        return X[:,u,:,:f_d].sum() + s[:,u,:,f_d:].sum() <= b
    def daily_limitation (self, k, X, u, d):
        return X[:,u,:,d].sum() <= k
    def campaign_limitation_rh(self, l_c, X, s, c, u, f_d):
#        print(f"X[{c},{u},:,:{f_d}].sum() + s[{c},{u},:,{f_d}:].sum() <={l_c[c]}")
        return X[c,u,:,:f_d].sum() + s[c,u,:,f_d:].sum() <=l_c[c]
    def weekly_quota(self, m_i, q_ic, X, u):
        return all((q_ic * X[:,u,:,:].sum(axis=(1,2))).sum(axis=1)<=m_i)
    def weekly_quota_rh(self, m_i, q_ic, X, s, u, f_d):
        print(f"X[:,{u},:,:{f_d}].sum() + s[:,{u},:,{f_d}:].sum() <={m_i}")
        return all((q_ic * X[:,u,:,:f_d].sum(axis=(1,2))).sum(axis=1) + (q_ic * s[:,u,:,:f_d].sum(axis=(1,2))).sum(axis=1)<=m_i)
    def daily_quota(self, n_i, q_ic, X, u, d):
        return all((q_ic * X[:,u,:,d].sum(axis=(1))).sum(axis=1)<=n_i)
    def channel_capacity(self, t_hd, X, h, d):
        return X[:,:,h,d].sum() <= t_hd[h,d]

    def check(self, X, PMS, indicies):
        if not self.eligibility(PMS.e_cu, X, indicies[self.c_i],indicies[self.u_i],indicies[self.h_i],indicies[self.d_i]):
#            print(f"eligibility => {indicies}")
            return False
#        for f_d in range(1, PMS.cuhd[self.d_i]+1):
#            if not self.weekly_limitation_rh(PMS.b, X, PMS.s_cuhd, indicies[self.u_i], f_d):
##                print(f"weekly_limitation_rh => {indicies}, {f_d}")
#                return False
        if not self.daily_limitation(PMS.k, X, indicies[self.u_i],indicies[self.d_i]):
#            print(f"daily_limitation => {indicies}")
            return False
#        for f_d in range(1, PMS.cuhd[self.d_i]+1):
#            if not self.campaign_limitation_rh(PMS.l_c, X, PMS.s_cuhd, indicies[self.c_i],indicies[self.u_i], f_d):
##                print(f"campaign_limitation_rh => {indicies}, {f_d}")
#                return False
        for f_d in range(1, PMS.cuhd[self.d_i]+1):
            if not self.weekly_quota_rh(PMS.m_i, PMS.q_ic, X, PMS.s_cuhd, indicies[self.u_i], f_d):
                print(f"weekly_quota_rh => {indicies}, {f_d}")
                return False
        if not self.daily_quota(PMS.n_i, PMS.q_ic, X, indicies[self.u_i],indicies[self.d_i]):
#            print(f"daily_quota => {indicies}")
            return False
        if not self.channel_capacity(PMS.t_hd, X, indicies[self.h_i],indicies[self.d_i]):
#            print(f"channel_capacity => {indicies}")
            return False
        return True


In [89]:
s_cuhd = np.zeros((C,U,H,D))
s_cuhd[:,:,:,D-1:D] = 1

In [76]:
u=0
f_d = 1

In [75]:
np.arange(1,7).reshape((2,3,1))

array([[[1],
        [2],
        [3]],

       [[4],
        [5],
        [6]]])

In [79]:
X_cuhd[:,0,:,:f_d] = np.arange(1,16).reshape((5,3,1))

In [81]:
(q_ic.shape, X_cuhd[:,u,:,:f_d].shape)

((3, 5), (5, 3, 1))

In [84]:
(q_ic, X_cuhd[:,u,:,:f_d])

(array([[1, 0, 0, 1, 0],
        [0, 1, 1, 1, 1],
        [1, 0, 0, 0, 1]]),
 array([[[ 1],
         [ 2],
         [ 3]],
 
        [[ 4],
         [ 5],
         [ 6]],
 
        [[ 7],
         [ 8],
         [ 9]],
 
        [[10],
         [11],
         [12]],
 
        [[13],
         [14],
         [15]]]))

In [101]:
X_cuhd[:,u,:,:f_d].sum(axis=(1,2))

array([ 6, 15, 24, 33, 42])

In [105]:
q_ic * s_cuhd[:,u,:,:f_d].sum(axis=(1,2))

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [104]:
(q_ic * X_cuhd[:,u,:,:f_d].sum(axis=(1,2)) + q_ic * s_cuhd[:,u,:,:f_d].sum(axis=(1,2)))

array([[ 6.,  0.,  0., 33.,  0.],
       [ 0., 15., 24., 33., 42.],
       [ 6.,  0.,  0.,  0., 42.]])

In [100]:
(q_ic * X_cuhd[:,u,:,:f_d].sum(axis=(1,2))).sum(axis=1)

array([ 39, 114,  48])

In [129]:
(q_ic * s_cuhd[:,u,:,:f_d].sum(axis=(1,2))).sum(axis=1)

array([0., 0., 0.])

In [147]:
(q_ic * X_cuhd[:,u,:,:f_d].sum(axis=(1,2))).sum(axis=1) + (q_ic * s_cuhd[:,u,:,f_d:].sum(axis=(1,2))).sum(axis=1)

array([ 45., 126.,  54.])

In [136]:
(X_cuhd[:,u,:,:f_d].sum(axis=(1,2)))

array([ 6, 15, 24, 33, 42])

In [119]:
m_i

array([4, 5, 3])

In [164]:
[(q_ic * X_cuhd[:,u,:,:f_d].sum(axis=(1,2))).sum(axis=1) + (q_ic * s_cuhd[:,u,:,f_d:].sum(axis=(1,2))).sum(axis=1)
for u in range(0,U)]

[array([ 45., 126.,  54.]),
 array([ 6., 12.,  6.]),
 array([ 6., 12.,  6.]),
 array([ 6., 12.,  6.]),
 array([ 6., 12.,  6.]),
 array([ 6., 12.,  6.]),
 array([ 6., 12.,  6.]),
 array([ 6., 12.,  6.]),
 array([ 6., 12.,  6.]),
 array([ 6., 12.,  6.]),
 array([ 6., 12.,  6.]),
 array([ 6., 12.,  6.]),
 array([ 6., 12.,  6.]),
 array([ 6., 12.,  6.]),
 array([ 6., 12.,  6.]),
 array([ 6., 12.,  6.]),
 array([ 6., 12.,  6.]),
 array([ 6., 12.,  6.]),
 array([ 6., 12.,  6.]),
 array([ 6., 12.,  6.]),
 array([ 6., 12.,  6.]),
 array([ 6., 12.,  6.]),
 array([ 6., 12.,  6.]),
 array([ 6., 12.,  6.]),
 array([ 6., 12.,  6.]),
 array([ 6., 12.,  6.]),
 array([ 6., 12.,  6.]),
 array([ 6., 12.,  6.]),
 array([ 6., 12.,  6.]),
 array([ 6., 12.,  6.]),
 array([ 6., 12.,  6.]),
 array([ 6., 12.,  6.]),
 array([ 6., 12.,  6.]),
 array([ 6., 12.,  6.]),
 array([ 6., 12.,  6.]),
 array([ 6., 12.,  6.]),
 array([ 6., 12.,  6.]),
 array([ 6., 12.,  6.]),
 array([ 6., 12.,  6.]),
 array([ 6., 12.,  6.]

In [151]:
for c in (
            (mdl.sum( (X_cuhd[(c,u,h,d)] if d < f_d else s_cuhd[(c,u,h,d)])* q_ic[i,c]
                for d in range(0,D)
                for c in range(0,C)
                for h in range(0,H)) <= m_i[i])
            for i in range(0,I)
            for u in range(0,U)):
    print(c)

45 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
6 <= 4
126 <= 5
12 <= 5
12 <= 5
12 <= 5
12 <= 5
12 <= 5
12 <= 5
12 <= 5
12 <= 5
12 <= 5
12 <= 5
12 <= 5
12 <= 5
12 <= 5
12 <= 5
12 <= 5
12 <= 5
12 <= 5
12 <= 5
12 <= 5
12 <= 5
12 <= 5
12 <= 5
12 <= 5
12 <= 5
12 <= 5
12 <= 5
12 <= 5
12 <= 5
12 <= 5
12 <= 5
12 <= 5
12 <= 5
12 <= 5
12 <= 5
12 <= 5
12 <= 5
12